In [4]:
# 引用套件
import boto3

# 建立與 sqs 的連線
client = boto3.client(
    'sqs',
    endpoint_url = 'http://cc104.sqs.local:9324',
    region_name = 'dummy_region',
    aws_access_key_id = 'dummy_access_key',
    aws_secret_access_key = 'dummy_secret_key',
    verify = False)

In [5]:
# 建立 queue
client.create_queue(QueueName='test_queue')

{'QueueUrl': 'http://cc104.sqs.local:9324/queue/test_queue',
 'ResponseMetadata': {'RequestId': '00000000-0000-0000-0000-000000000000',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'akka-http/10.0.2',
   'date': 'Wed, 16 Jan 2019 11:47:01 GMT',
   'content-type': 'text/plain; charset=UTF-8',
   'content-length': '429'},
  'RetryAttempts': 0}}

In [8]:
# 調用 test_queue
# get a list of queues, we get back a dict with 'QueueUrls' as a key with a list of queue URLs
# we filter to narrow down the list
queues = client.list_queues(QueueNamePrefix='test_queue')
test_queue_url = queues['QueueUrls'][0]
print(test_queue_url)

http://cc104.sqs.local:9324/queue/test_queue


In [4]:
# 寫入訊息
# send 10 messages to this queue
for i in range(0,10):
    # we set a simple message body for each message
    # for FIFO queues, a 'MessageGroupId' is required, which is a 128 char alphanumeric string
    enqueue_response = client.send_message(
        QueueUrl = test_queue_url, 
        MessageBody = 'This is test message #'+str(i)
    )
    # the response contains MD5 of the body, a message Id, MD5 of message attributes, and a sequence number (for FIFO queues)
    print('Message ID : ',enqueue_response['MessageId'])

Message ID :  f3d22cc7-61ee-432f-ac2e-6ecd0824cced
Message ID :  53ff0d29-7a8e-44ef-abde-55d85cdddfc4
Message ID :  853f7755-790d-4b63-a087-05f7310ab087
Message ID :  89920b64-7438-400e-b845-b51b484784ba
Message ID :  a365d63a-fb75-4e46-8324-067536434a5f
Message ID :  98df9eee-a9fd-4b44-a0ac-e99f7dcaa9b9
Message ID :  11c64adb-f139-4f91-8f15-51484b101f67
Message ID :  2082143f-dfbf-43aa-801c-cdeb8fcb5360
Message ID :  16aefb9a-5dfe-4e87-8708-3abe9213716c
Message ID :  0ea997e9-87bc-4a43-8a53-45ae52a0d12f


In [5]:
# 刪除訊息
# next, we dequeue these messages - 10 messages at a time (SQS max limit) till the queue is exhausted.
# in production/real setup, I suggest using long polling as you get billed for each request, regardless of an empty response
while True:
    messages = client.receive_message(QueueUrl=test_queue_url,MaxNumberOfMessages=10) # adjust MaxNumberOfMessages if needed
    if 'Messages' in messages: # when the queue is exhausted, the response dict contains no 'Messages' key
        for message in messages['Messages']: # 'Messages' is a list
            # process the messages
            print(message['Body'])
            # next, we delete the message from the queue so no one else will process it again
            client.delete_message(
                QueueUrl=test_queue_url,
                ReceiptHandle=message['ReceiptHandle']
            )
    else:
        print('Queue is now empty')
        break

This is test message #9
This is test message #8
This is test message #7
This is test message #6
This is test message #5
This is test message #4
This is test message #3
This is test message #2
This is test message #1
This is test message #0
Queue is now empty


In [6]:
# 刪除 test queue
# delete queue
client.delete_queue(QueueUrl=test_queue_url)

{'ResponseMetadata': {'RequestId': '00000000-0000-0000-0000-000000000000',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'akka-http/10.0.2',
   'date': 'Sun, 13 Jan 2019 07:04:53 GMT',
   'content-type': 'text/plain; charset=UTF-8',
   'content-length': '250'},
  'RetryAttempts': 0}}